## MODELOS

# Carga de Dataframes

In [9]:
import cv2
import numpy
import pandas
import imageio.v3 as io
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from tqdm import tqdm
from typing import Optional, Union

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Datos tabulares train, test, validation.

In [3]:
x_train_data = pandas.read_csv("/content/drive/MyDrive/x_train_data.csv", sep=';')
x_test_data = pandas.read_csv("/content/drive/MyDrive/x_test_data.csv", sep=';')
x_validation_data = pandas.read_csv("/content/drive/MyDrive/x_validation_data.csv", sep=';')

x_train_data.shape, x_test_data.shape, x_validation_data.shape

((2317, 21), (1531, 21), (775, 21))

Datos tabulares variable objetivo 'Price' para regresión.

In [4]:
y_reg_train = pandas.read_csv("/content/drive/MyDrive/y_reg_train.csv", sep=';')
y_reg_test = pandas.read_csv("/content/drive/MyDrive/y_reg_test.csv", sep=';')
y_reg_validation = pandas.read_csv("/content/drive/MyDrive/y_reg_validation.csv", sep=';')

y_reg_train.shape, y_reg_test.shape, y_reg_validation.shape

((2317, 1), (1531, 1), (775, 1))

Datos tabulares variable objetivo 'Price' para clasificación.

In [5]:
y_class_train = pandas.read_csv("/content/drive/MyDrive/y_class_train.csv", sep=';')
y_class_test = pandas.read_csv("/content/drive/MyDrive/y_class_test.csv", sep=';')
y_class_validation = pandas.read_csv("/content/drive/MyDrive/y_class_validation.csv", sep=';')

y_class_train.shape, y_class_test.shape, y_class_validation.shape

((2317, 3), (1531, 3), (775, 3))

Imágenes train, test y validation.

In [6]:
images_train = numpy.load("/content/drive/MyDrive/images_train.npy")
images_test = numpy.load("/content/drive/MyDrive/images_test.npy")
images_validation = numpy.load("/content/drive/MyDrive/images_validation.npy")

images_train.shape, images_test.shape, images_validation.shape

((2317, 224, 224, 3), (1531, 224, 224, 3), (775, 224, 224, 3))

 Se transforma los dataframes.

In [7]:
x_train_data = x_train_data.values
x_test_data = x_test_data.values
x_validation_data = x_validation_data.values

y_reg_train = y_reg_train.values
y_reg_test = y_reg_test.values
y_reg_validation = y_reg_validation.values

y_reg_train = numpy.log10(y_reg_train)
y_reg_test = numpy.log10(y_reg_test)
y_reg_validation = numpy.log10(y_reg_validation)

y_class_train = y_class_train.values
y_class_test = y_class_test.values
y_class_validation = y_class_validation.values

Se comprueba.

In [8]:
print(type(x_train_data))
print(type(x_test_data))
print(type(x_validation_data))

print(type(y_reg_train))
print(type(y_reg_test))
print(type(y_reg_validation))

print(type(y_class_train))
print(type(y_class_test))
print(type(y_class_validation))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## MODELO DE REGRSIÓN

# Modelo de Regresión datos numéricos

In [3]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

Se crea un modelo de red neuronal secuencial Dense. La primera capa contiene 16 neuronas y la función de activación es ReLu que se aplica a cada neurona. La capa recibe de entrada las variables de los datos de entrenamiento. En la segunda y tercera capa se utiliza la misma función de activación; no es necesario pasarle el tamaño de entrada ya que lo infiere directamente de la capa anterior. La última capa es una salida lineal para la predicción.

In [13]:
model = Sequential()
model.add (Dense (16, input_shape=(x_train_data.shape[1],), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='linear'))

Se compila. La funcion de pérdida se establece en 'mean_squared_error', que calcula el error cuadrático medio entre las predicciones del modelo y los valores reales de los datos de train. Se elige 'Adam' como optimizador ya que adapta automáticamente la tasa de aprendizaje durante el entrenamiento para cada parámetro de la red neuronal.

In [14]:
model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(x=x_train_data, y=y_reg_train,
	    validation_data=(x_validation_data, y_reg_validation),
	    epochs=50, batch_size=16)

Epoch 1/50
145/145 [==============================] - 1s 4ms/step - loss: 7.4585 - val_loss: 6.3614
Epoch 2/50
145/145 [==============================] - 0s 2ms/step - loss: 1.2515 - val_loss: 1.0268
Epoch 3/50
145/145 [==============================] - 0s 2ms/step - loss: 0.6924 - val_loss: 2.8088
Epoch 4/50
145/145 [==============================] - 0s 2ms/step - loss: 0.4920 - val_loss: 1.7040
Epoch 5/50
145/145 [==============================] - 0s 2ms/step - loss: 0.3496 - val_loss: 6.6990
Epoch 6/50
145/145 [==============================] - 0s 3ms/step - loss: 0.3184 - val_loss: 12.3799
Epoch 7/50
145/145 [==============================] - 1s 4ms/step - loss: 0.2821 - val_loss: 7.4059
Epoch 8/50
145/145 [==============================] - 1s 4ms/step - loss: 0.2380 - val_loss: 4.0910
Epoch 9/50
145/145 [==============================] - 0s 3ms/step - loss: 0.2739 - val_loss: 6.3869
Epoch 10/50
145/145 [==============================] - 0s 3ms/step - loss: 0.1825 - val_loss: 4.528

Tanto la pérdida de entrenamiento 'loss' como la pérdida de validación 'val_loss' disminuyen con cada época, lo que indica que el modelo está aprendiendo y mejorando su capacidad para predecir tanto en los datos de entrenamiento como en los de validación.

A lo largo de todas las épocas, la pérdida de entrenamiento es menor que la pérdida de validación, lo que sugiere que el modelo puede estar sobreajustando los datos de entrenamiento. Aunque el loss sigue disminuyendo, el val_loss comienza a aumentar después de la época 20, lo que indica que el modelo deja de aprender a partir de esa época y comienza a sobreajustarse. Por lo tanto, se puede considerar que el modelo no aprende de manera efectiva después de la época 20. La diferencia creciente entre el loss y val_loss también indica la presencia de sobreajuste, lo que sugiere la necesidad de técnicas de regularización o ajuste de hiperparámetros para mejorar la generalización del modelo.

Sin embargo, la diferencia entre ambas pérdidas parece mantenerse relativamente constante a lo largo del entrenamiento. A partir de alrededor de la época 40, la disminución en la pérdida tanto de entrenamiento como de validación es mínima. Esto sugiere que el modelo ya ha aprendido la mayoría de los patrones importantes en los datos y que continuar entrenando más allá de este punto probablemente no conducirá a mejoras significativas en el desempeño del modelo.

Se comprueba el modelo con los datos de test.

In [15]:
scores = model.evaluate(x_test_data, y_reg_test, verbose=0)
print('Loss: %.3f' % scores)

Loss: 0.074


El hecho de que la pérdida en el conjunto de pruebas sea cercana a la pérdida en el conjunto de entrenamiento (0.074 en test y alrededor de 0.059 en train) sugiere que el modelo no está sufriendo de sobreajuste (overfitting), al menos en términos de pérdida.

Se prueba la red neuronal con la regularización Droput=0.5, lo que significa que se desactivan la mitad de las neuronas en cada interación de entrenamiento.

In [16]:
from keras.layers import Dropout
model_dp = Sequential()
model_dp.add(Dense(16, input_shape=(x_train_data.shape[1],), activation='relu'))
model_dp.add(Dropout(0.5))
model_dp.add(Dense(8, activation='relu'))
model_dp.add(Dropout(0.5))
model_dp.add(Dense(4, activation='relu'))
model_dp.add(Dropout(0.5))
model_dp.add(Dense(1, activation='linear'))

In [17]:
model_dp.compile(loss='mean_squared_error', optimizer='adam')

model_dp.fit(x=x_train_data, y=y_reg_train,
          validation_data=(x_validation_data, y_reg_validation),
          epochs=50, batch_size=16)

Epoch 1/50
145/145 [==============================] - 2s 3ms/step - loss: 171042.7969 - val_loss: 14267.6953
Epoch 2/50
145/145 [==============================] - 0s 2ms/step - loss: 34258.8164 - val_loss: 1460.6324
Epoch 3/50
145/145 [==============================] - 0s 2ms/step - loss: 15796.1748 - val_loss: 240.2660
Epoch 4/50
145/145 [==============================] - 0s 2ms/step - loss: 8425.3848 - val_loss: 85.4376
Epoch 5/50
145/145 [==============================] - 0s 2ms/step - loss: 3667.7722 - val_loss: 109.3726
Epoch 6/50
145/145 [==============================] - 0s 2ms/step - loss: 2040.2085 - val_loss: 60.9684
Epoch 7/50
145/145 [==============================] - 0s 2ms/step - loss: 1881.7991 - val_loss: 129.4271
Epoch 8/50
145/145 [==============================] - 0s 2ms/step - loss: 1001.9824 - val_loss: 192.9594
Epoch 9/50
145/145 [==============================] - 0s 2ms/step - loss: 697.3353 - val_loss: 264.3174
Epoch 10/50
145/145 [==============================

In [18]:
scores = model_dp.evaluate(x_test_data, y_reg_test, verbose=0)
print('Loss: %.3f' % scores)

Loss: 0.152


No se aprecia mejoria. Se optimiza el hiperparámetro Learning_rate.

In [20]:
model_lr = Sequential()
model_lr.add(Dense(16, input_shape=(x_train_data.shape[1],), activation='relu'))
model_lr.add(Dense(8, activation='relu'))
model_lr.add(Dense(4, activation='relu'))
model_lr.add(Dense(1, activation='linear'))

optimizer = Adam(learning_rate=0.001)
model_lr.compile(loss='mean_squared_error', optimizer=optimizer)

model_lr.fit(x=x_train_data, y=y_reg_train,
          validation_data=(x_validation_data, y_reg_validation),
          epochs=50, batch_size=32)

Epoch 1/50
73/73 [==============================] - 1s 5ms/step - loss: 345.9076 - val_loss: 6.6354
Epoch 2/50
73/73 [==============================] - 0s 2ms/step - loss: 3.6250 - val_loss: 2.1532
Epoch 3/50
73/73 [==============================] - 0s 3ms/step - loss: 1.4915 - val_loss: 1.3554
Epoch 4/50
73/73 [==============================] - 0s 3ms/step - loss: 1.0765 - val_loss: 1.0812
Epoch 5/50
73/73 [==============================] - 0s 2ms/step - loss: 0.9558 - val_loss: 0.9163
Epoch 6/50
73/73 [==============================] - 0s 2ms/step - loss: 0.8899 - val_loss: 0.8486
Epoch 7/50
73/73 [==============================] - 0s 3ms/step - loss: 0.8461 - val_loss: 0.7895
Epoch 8/50
73/73 [==============================] - 0s 3ms/step - loss: 0.8100 - val_loss: 0.7242
Epoch 9/50
73/73 [==============================] - 0s 3ms/step - loss: 0.7747 - val_loss: 0.6925
Epoch 10/50
73/73 [==============================] - 0s 3ms/step - loss: 0.7410 - val_loss: 0.6498
Epoch 11/50
73/73

In [21]:
scores = model_lr.evaluate(x_test_data, y_reg_test, verbose=0)
print('Loss: %.3f' % scores)

Loss: 0.098


En base a los resultados y la ausencia de señales claras de sobreajuste, parece que el modelo de regresión lineal está generalizando bien y proporcionando resultados sólidos tanto en los datos de validación como en los datos de prueba.

# Modelo de Regresión con imágenes

Se implementa un modelo de regresión neuronal utilizando Keras con TensorFlow. Para desarrollar un sistema capaz de predecir valores numéricos a partir de imágenes se utiliza una arquitectura de red convolucional (CNN) que tomará las imágenes comom entrada y generará una salida numérica.

La arquitectura consta de varias capas, incluidas capas convolucionales, de agrupación y densas.

Se utiliza el optimizador Adam para minimizar la función de perdida del error cuadrático medio durante el entrenamiento.

In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam

In [26]:
inputShape = (224, 224, 3)
chanDim = -1

filters=(16, 32, 64)
inputs = Input(shape=inputShape)

for (i, f) in enumerate(filters):

    if i == 0:
        x = inputs

    x = Conv2D(f, (3, 3), padding="same")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(16)(x)
x = BatchNormalization(axis=chanDim)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
x = Dense(8)(x)
x = Activation("relu")(x)
x = Dense(4)(x)
x = Activation("relu")(x)
x = Dense(1, activation="linear")(x)

model_img_reg = Model(inputs, x)

optim = Adam(lr=1e-3)
model_img_reg .compile(loss="mean_squared_error", optimizer=optim)

model_img_reg .fit(x=images_train, y=y_reg_train,
    validation_data=(images_validation, y_reg_validation),
    epochs=50, batch_size=8)

Epoch 1/50
290/290 [==============================] - 15s 18ms/step - loss: 0.3610 - val_loss: 0.0773
Epoch 2/50
290/290 [==============================] - 4s 13ms/step - loss: 0.1326 - val_loss: 0.0764
Epoch 3/50
290/290 [==============================] - 4s 14ms/step - loss: 0.0977 - val_loss: 0.1126
Epoch 4/50
290/290 [==============================] - 4s 13ms/step - loss: 0.0844 - val_loss: 0.0967
Epoch 5/50
290/290 [==============================] - 4s 13ms/step - loss: 0.0796 - val_loss: 0.0886
Epoch 6/50
290/290 [==============================] - 4s 14ms/step - loss: 0.0756 - val_loss: 0.1105
Epoch 7/50
290/290 [==============================] - 4s 13ms/step - loss: 0.0732 - val_loss: 0.0989
Epoch 8/50
290/290 [==============================] - 4s 13ms/step - loss: 0.0726 - val_loss: 0.1054
Epoch 9/50
290/290 [==============================] - 4s 13ms/step - loss: 0.0712 - val_loss: 0.0818
Epoch 10/50
290/290 [==============================] - 4s 14ms/step - loss: 0.0711 - val_l

In [27]:
scores = model_img_reg.evaluate(images_test, y_reg_test, verbose=0)
print('Loss: %.3f' % scores)

Loss: 0.190


Parece que el modelo de regresión neuronal para imágenes está aprendiendo de manera efectiva y generaliza bien en datos no vistos, lo que es un buen indicador de que es un modelo útil.

# Modelo de Regresión con datos e imágenes.

Se van a combinar dos modelos de redes neuronales, uno de datos numéricos y otro para imágenes utilizando la concatenación.  El modelo de datos se realiza con capas densas con activación y para las imágenes se utilizan capas convolucionales y normalización por lotes con activación. Cuando se concatenan se agregan capas densas adicionales para las predicciones finales. Se utiliza como optimizador Adam.

In [43]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input

In [44]:
model_mix_reg = Sequential()
model_mix_reg .add(Dense(16, input_dim=x_train_data.shape[1], activation="relu"))
model_mix_reg .add(Dense(8, input_dim=x_train_data.shape[1], activation="relu"))
model_mix_reg .add(Dense(4, activation="relu"))

In [45]:
inputShape = (224, 224, 3)
chanDim = -1

filters=(16, 32, 64)
inputs = Input(shape=inputShape)

for (i, f) in enumerate(filters):
  if i == 0:
	  x = inputs

  x = Conv2D(f, (3, 3), padding="same")(x)
  x = BatchNormalization(axis=chanDim)(x)
  x = Activation("relu")(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(16)(x)
x = BatchNormalization(axis=chanDim)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
x = Dense(8)(x)
x = Activation("relu")(x)
x = Dense(4)(x)
x = Activation("relu")(x)

cnn = Model(inputs, x)

Se combinan ambas redes neuronales

In [46]:
from tensorflow.keras.layers import concatenate

combinedInput = concatenate([model_mix_reg.output, cnn.output])

In [56]:
from tensorflow.keras.optimizers import Adam

x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

model_final_reg = Model(inputs=[model_mix_reg.input, cnn.input], outputs=x)

optim = Adam(lr=1e-3)
model_final_reg.compile(loss="mean_squared_error", optimizer=optim)

model_final_reg.fit(
	x=[x_train_data, images_train], y=y_reg_train,
	validation_data=([x_validation_data, images_validation], y_reg_validation),
	epochs=50, batch_size=8)


Epoch 1/50
290/290 [==============================] - 10s 18ms/step - loss: 4.5842 - val_loss: 5.2876
Epoch 2/50
290/290 [==============================] - 4s 14ms/step - loss: 0.4399 - val_loss: 1.8936
Epoch 3/50
290/290 [==============================] - 4s 15ms/step - loss: 0.2682 - val_loss: 0.2163
Epoch 4/50
290/290 [==============================] - 4s 14ms/step - loss: 0.1762 - val_loss: 0.1407
Epoch 5/50
290/290 [==============================] - 4s 14ms/step - loss: 0.1304 - val_loss: 0.1205
Epoch 6/50
290/290 [==============================] - 4s 15ms/step - loss: 0.1012 - val_loss: 0.1188
Epoch 7/50
290/290 [==============================] - 4s 14ms/step - loss: 0.0858 - val_loss: 0.0917
Epoch 8/50
290/290 [==============================] - 4s 14ms/step - loss: 0.0793 - val_loss: 0.0776
Epoch 9/50
290/290 [==============================] - 4s 15ms/step - loss: 0.0766 - val_loss: 0.0762
Epoch 10/50
290/290 [==============================] - 4s 14ms/step - loss: 0.0750 - val_l

In [58]:
scores = model_final_reg.evaluate([x_test_data, images_test], y_reg_test, verbose=0)
print('Loss: %.3f' % scores)

Loss: 0.086


En el modelo se aprecia una disminución gradual de loss lo que indica que el aprendizaje es efectivo. Pero se observa una brecha entre loss y val_loss. La pérdida en test es aceptable pero podría mejorarse.

## MODELO DE CLASIFICACIÓN

# Modelo clasificación con datos numéricos

Modelo de clasificador de datos tabulares con cuatro capas densas, se utiliza la función de pérdida de crossentropy para la optimización durante el entrenamiento.

In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [69]:
model_class_1 = Sequential(name="Clasificador_datos_tabulares")
model_class_1 .add(Dense(16, input_shape=(21,), activation='relu'))
model_class_1 .add(Dense(8, activation='relu'))
model_class_1 .add(Dense(4, activation='relu'))
model_class_1 .add(Dense(3, activation='softmax'))

In [70]:
model_class_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [72]:
history1 = model_class_1.fit(x=x_train_data, y=y_class_train, batch_size=32, epochs=10, validation_data=(x_validation_data, y_class_validation))


Epoch 1/10
73/73 [==============================] - 0s 4ms/step - loss: 1.0407 - accuracy: 0.4635 - val_loss: 1.0578 - val_accuracy: 0.4555
Epoch 2/10
73/73 [==============================] - 0s 4ms/step - loss: 1.0402 - accuracy: 0.4635 - val_loss: 1.0609 - val_accuracy: 0.4555
Epoch 3/10
73/73 [==============================] - 0s 4ms/step - loss: 1.0420 - accuracy: 0.4635 - val_loss: 1.0542 - val_accuracy: 0.4555
Epoch 4/10
73/73 [==============================] - 0s 4ms/step - loss: 1.0397 - accuracy: 0.4635 - val_loss: 1.0567 - val_accuracy: 0.4555
Epoch 5/10
73/73 [==============================] - 0s 4ms/step - loss: 1.0386 - accuracy: 0.4635 - val_loss: 1.0577 - val_accuracy: 0.4555
Epoch 6/10
73/73 [==============================] - 0s 4ms/step - loss: 1.0382 - accuracy: 0.4635 - val_loss: 1.0506 - val_accuracy: 0.4542
Epoch 7/10
73/73 [==============================] - 0s 4ms/step - loss: 1.0300 - accuracy: 0.4635 - val_loss: 1.0630 - val_accuracy: 0.4542
Epoch 8/10
73/73 [==

In [73]:
test_loss, test_accuracy = model_class_1.evaluate(x_test_data, y_class_test, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

48/48 [==============================] - 0s 3ms/step - loss: 1.0258 - accuracy: 0.4559
Test Loss: 1.0258113145828247
Test Accuracy: 0.45591115951538086


El modelo entrenado no es capaz de capturar de manera efectiva los patrones en los datos para realizar una clasificación precisa, ya que tanto la precisión en el conjunto de validación como en el conjunto de prueba son relativamente bajas y no muestran signos de mejora significativa durante el entrenamiento.

Se ajusta la arquitectura del modelo para incluir una capa adicional de 64 neuronas con una capa de regularización Dropout para reducir el sobreajuste. También se ajustan los hiperparámetros de entrenamiento Epochs y Batch_size.

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [18]:
model_class_2 = Sequential(name="Clasificador_datos_tabulares")
model_class_2.add(Dense(64, input_shape=(x_train_data.shape[1],), activation='relu'))
model_class_2.add(Dense(32, activation='relu'))
model_class_2.add(Dropout(0.5))
model_class_2.add(Dense(8, activation='relu'))
model_class_2.add(Dense(4, activation='relu'))
model_class_2.add(Dense(3, activation='softmax'))

In [19]:
model_class_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
history2 = model_class_2.fit(x=x_train_data,
                            y=y_class_train,
                            batch_size=32,
                            epochs=50,
                            validation_data=(x_validation_data, y_class_validation),
                            verbose=1)

Epoch 1/50
73/73 [==============================] - 1s 4ms/step - loss: 16.8085 - accuracy: 0.3591 - val_loss: 1.1051 - val_accuracy: 0.2529
Epoch 2/50
73/73 [==============================] - 0s 3ms/step - loss: 1.5779 - accuracy: 0.3543 - val_loss: 1.0976 - val_accuracy: 0.2942
Epoch 3/50
73/73 [==============================] - 0s 3ms/step - loss: 1.1931 - accuracy: 0.4221 - val_loss: 1.0897 - val_accuracy: 0.4594
Epoch 4/50
73/73 [==============================] - 0s 3ms/step - loss: 1.1117 - accuracy: 0.4549 - val_loss: 1.0808 - val_accuracy: 0.4529
Epoch 5/50
73/73 [==============================] - 0s 3ms/step - loss: 1.1078 - accuracy: 0.4614 - val_loss: 1.0750 - val_accuracy: 0.4529
Epoch 6/50
73/73 [==============================] - 0s 3ms/step - loss: 1.0879 - accuracy: 0.4653 - val_loss: 1.0695 - val_accuracy: 0.4529
Epoch 7/50
73/73 [==============================] - 0s 2ms/step - loss: 1.0775 - accuracy: 0.4635 - val_loss: 1.0682 - val_accuracy: 0.4542
Epoch 8/50
73/73 [=

In [21]:
test_loss, test_accuracy = model_class_2.evaluate(x_test_data, y_class_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

48/48 [==============================] - 0s 974us/step - loss: 1.0412 - accuracy: 0.4637
Test Loss: 1.0411802530288696
Test Accuracy: 0.4637491703033447


No han resultado las mejoras que se han realizado. Se prueba una regularización L2.

In [22]:
from keras.regularizers import l2

In [95]:
model_class_3 = Sequential(name="Clasificador_datos_tabulares")
model_class_3.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01), input_shape=(21,)))
model_class_3.add(Dense(8, activation='relu', kernel_regularizer=l2(0.01)))
model_class_3.add(Dense(4, activation='relu', kernel_regularizer=l2(0.01)))
model_class_3.add(Dense(3, activation='softmax'))

In [26]:
model_class_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
model_class_3.fit(x_train_data, y_class_train, epochs=50, batch_size=32, validation_data=(x_validation_data, y_class_validation))

Epoch 1/50
73/73 [==============================] - 1s 4ms/step - loss: 16.3356 - accuracy: 0.3815 - val_loss: 6.5751 - val_accuracy: 0.3768
Epoch 2/50
73/73 [==============================] - 0s 2ms/step - loss: 2.6932 - accuracy: 0.4342 - val_loss: 2.0259 - val_accuracy: 0.4826
Epoch 3/50
73/73 [==============================] - 0s 2ms/step - loss: 1.5966 - accuracy: 0.4661 - val_loss: 1.5024 - val_accuracy: 0.4684
Epoch 4/50
73/73 [==============================] - 0s 2ms/step - loss: 1.4302 - accuracy: 0.4627 - val_loss: 1.4195 - val_accuracy: 0.4555
Epoch 5/50
73/73 [==============================] - 0s 2ms/step - loss: 1.3816 - accuracy: 0.4657 - val_loss: 1.3650 - val_accuracy: 0.4555
Epoch 6/50
73/73 [==============================] - 0s 2ms/step - loss: 1.3520 - accuracy: 0.4661 - val_loss: 1.3305 - val_accuracy: 0.4568
Epoch 7/50
73/73 [==============================] - 0s 2ms/step - loss: 1.3326 - accuracy: 0.4657 - val_loss: 1.3099 - val_accuracy: 0.4581
Epoch 8/50
73/73 [=

In [28]:
test_loss, test_accuracy = model_class_3.evaluate(x_test_data, y_class_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

48/48 [==============================] - 0s 937us/step - loss: 1.1633 - accuracy: 0.4611
Test Loss: 1.1633301973342896
Test Accuracy: 0.46113651990890503


Parece que mejora gradualmente con el tiempo, lo que indica que el modelo esta aprendiendo de manera efectiva y no esta sobreajustado. Pero en el caso de los datos test indica que el modelo pordria optimizarse para una mejor generalización.

# Modelo clasificación con imágenes


Se va a implementar un modelo de red neuronal convolucional (CNN) utilizando la arquitectura VGG16preentrenada en el conjunto de datos ImageNet para la clasificación de imágenes. Se cargan los pesos entrenados excluyendo la capa superior (full connected) para adaptarla al problema. Se agregan capas adicionales, icluyendo una capa de aplanado (Flatten) seguido de capas densas (Full connected) con ativación ReLu y una capa de Dropout para la regularización utilizando finalmente softmax para la clasificación multiclase. Como optimizado se usa Adam con un learning_rate= 0.001.

In [25]:
from tensorflow.keras.applications import VGG16
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [27]:
model_VGG = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [33]:
model_img = Sequential()
model_img.add(model_VGG)
model_img.add(Flatten())
model_img.add(Dense(224, activation='relu'))
model_img.add(Dropout(0.5))
model_img.add(Dense(3, activation='softmax'))

In [34]:
for layer in model_VGG.layers:
    layer.trainable = False

In [36]:
model_img.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
history_img = model_img.fit(images_train, y_class_train, epochs=10, batch_size=32, validation_data=(images_validation, y_class_validation))

Epoch 1/10
73/73 [==============================] - 13s 169ms/step - loss: 7.8853 - accuracy: 0.4445 - val_loss: 1.0952 - val_accuracy: 0.5071
Epoch 2/10
73/73 [==============================] - 13s 185ms/step - loss: 1.0298 - accuracy: 0.5757 - val_loss: 1.0856 - val_accuracy: 0.4632
Epoch 3/10
73/73 [==============================] - 14s 187ms/step - loss: 0.9358 - accuracy: 0.5999 - val_loss: 1.0772 - val_accuracy: 0.4671
Epoch 4/10
73/73 [==============================] - 11s 156ms/step - loss: 0.8956 - accuracy: 0.6081 - val_loss: 1.0737 - val_accuracy: 0.4555
Epoch 5/10
73/73 [==============================] - 11s 156ms/step - loss: 0.8869 - accuracy: 0.6262 - val_loss: 1.0757 - val_accuracy: 0.4735
Epoch 6/10
73/73 [==============================] - 14s 188ms/step - loss: 0.7798 - accuracy: 0.6323 - val_loss: 1.0818 - val_accuracy: 0.4826
Epoch 7/10
73/73 [==============================] - 14s 188ms/step - loss: 0.7705 - accuracy: 0.6323 - val_loss: 1.0790 - val_accuracy: 0.4632

In [38]:
test_loss, test_accuracy = model_img.evaluate(images_test, y_class_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

48/48 [==============================] - 6s 120ms/step - loss: 1.0661 - accuracy: 0.4820
Test Loss: 1.0661242008209229
Test Accuracy: 0.48203787207603455


El modelo parece que esta aprendiendo de manera efectiva durante las primeras épocas, pero hay un punto en donde se estabiliza y no hay mejoras significativas, aunque tienen una precisión razonable en los datos de test, se podrian realizar algun ajuste.

Se prueba el modelo ResNet50 para mitigar el problema de la desaparición del gradiente mediante conexiones residuales a ver si da mejores resultados. Se van a agregar capas adicionale, incluyendo una capa de agrupación máxima (MaxPoling2D), una convolucional con activación ReLu y regularización L2.

In [87]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2

In [81]:
model_RN = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [82]:
model_RN.trainable = False

In [88]:
model_img_1 = Sequential()
model_img_1.add(model_RN)
model_img_1.add(MaxPooling2D(pool_size=(2, 2)))
model_img_1.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model_img_1.add(Flatten())
model_img_1.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
model_img_1.add(Dropout(0.5))
model_img_1.add(Dense(3, activation='softmax'))

In [89]:
model_img_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [90]:
history_img_1 = model_img_1.fit(images_train, y_class_train, epochs=10, validation_data=(images_validation, y_class_validation))

Epoch 1/10
73/73 [==============================] - 16s 151ms/step - loss: 2.9727 - accuracy: 0.4432 - val_loss: 2.2291 - val_accuracy: 0.4555
Epoch 2/10
73/73 [==============================] - 9s 123ms/step - loss: 1.8941 - accuracy: 0.4955 - val_loss: 1.8040 - val_accuracy: 0.2542
Epoch 3/10
73/73 [==============================] - 9s 119ms/step - loss: 1.5484 - accuracy: 0.5183 - val_loss: 1.6132 - val_accuracy: 0.2542
Epoch 4/10
73/73 [==============================] - 9s 119ms/step - loss: 1.3396 - accuracy: 0.5585 - val_loss: 1.5421 - val_accuracy: 0.2542
Epoch 5/10
73/73 [==============================] - 9s 124ms/step - loss: 1.2267 - accuracy: 0.5839 - val_loss: 2.1311 - val_accuracy: 0.4555
Epoch 6/10
73/73 [==============================] - 9s 125ms/step - loss: 1.0720 - accuracy: 0.6206 - val_loss: 2.1116 - val_accuracy: 0.4555
Epoch 7/10
73/73 [==============================] - 9s 120ms/step - loss: 0.9798 - accuracy: 0.6396 - val_loss: 2.1683 - val_accuracy: 0.4555
Epoch

In [91]:
test_loss, test_accuracy = model_img_1.evaluate(images_test, y_class_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

48/48 [==============================] - 5s 98ms/step - loss: 3.0228 - accuracy: 0.4559
Test Loss: 3.0227842330932617
Test Accuracy: 0.45591115951538086


El modelo no esta aprendiendo de manera efectiva, tenemos mejores resultados con el modelo anterior.

# Modelo clasificación datos e imágenes

Se implementa un modelo Late Fusion combinando los modelos previamente entrenados model_class_3 para datos numéricos y model_img para imágenes.

Se definen las entradas para ambos modelos input_numeric y input_image y las salidas output_numeric y output_image. Ambas salidas se fusionan utilizando la capa concatenate. Se agregan capas densas para procesar la información fusionada aplicando regularización L2 y Dropout para evitar el sobreajuste. Se utiliza el optimizador Adam y crossentropy.

In [121]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate, Input, Dense, Dropout
from tensorflow.keras.regularizers import l2

In [122]:
input_numeric = Input(shape=(21,))
input_image = Input(shape=(224, 224, 3))

In [123]:
output_numeric = model_class_3(input_numeric)
output_image = model_img(input_image)

In [125]:
merged_output = concatenate([output_numeric, output_image])

In [126]:
fusion_layer_1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(merged_output)
fusion_layer_1_dropout = Dropout(0.5)(fusion_layer_1)
fusion_layer_2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(fusion_layer_1)
fusion_layer_1_dropout = Dropout(0.5)(fusion_layer_1)
fusion_layer_3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(fusion_layer_2)

In [127]:
output = Dense(3, activation='softmax')(fusion_layer_3)

In [128]:
late_fusion_model = Model(inputs=[input_numeric, input_image], outputs=output)

In [129]:
late_fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [130]:
history_fusion = late_fusion_model.fit([x_train_data, images_train], y_class_train, epochs=10, batch_size=32, validation_data=([x_validation_data, images_validation], y_class_validation))


Epoch 1/10
73/73 [==============================] - 19s 210ms/step - loss: 1.9271 - accuracy: 0.5222 - val_loss: 1.5295 - val_accuracy: 0.5652
Epoch 2/10
73/73 [==============================] - 11s 154ms/step - loss: 1.2584 - accuracy: 0.5999 - val_loss: 1.1926 - val_accuracy: 0.5677
Epoch 3/10
73/73 [==============================] - 13s 186ms/step - loss: 1.0515 - accuracy: 0.6155 - val_loss: 1.0430 - val_accuracy: 0.5716
Epoch 4/10
73/73 [==============================] - 11s 156ms/step - loss: 0.9835 - accuracy: 0.6219 - val_loss: 1.0437 - val_accuracy: 0.5755
Epoch 5/10
73/73 [==============================] - 14s 187ms/step - loss: 0.9312 - accuracy: 0.6280 - val_loss: 1.0769 - val_accuracy: 0.5303
Epoch 6/10
73/73 [==============================] - 11s 157ms/step - loss: 0.9165 - accuracy: 0.6383 - val_loss: 0.9920 - val_accuracy: 0.5832
Epoch 7/10
73/73 [==============================] - 14s 188ms/step - loss: 0.9238 - accuracy: 0.6362 - val_loss: 0.9602 - val_accuracy: 0.6103

In [131]:
test_loss, test_accuracy = late_fusion_model.evaluate([x_test_data, images_test], y_class_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

48/48 [==============================] - 6s 120ms/step - loss: 0.9103 - accuracy: 0.6055
Test Loss: 0.9102519154548645
Test Accuracy: 0.6054866313934326


El modelo logra una precisión buena, está generalizando adecuadamente.